# Running Instructions

Below instructions tested in Ubuntu 22.04.3 (Windows Subsystem for Linux 2, WSL2).  
1. Create a clean Python 3.10 virtual environment: `python -m venv env_planted_solutions`
2. Activate environment: `source env_run_dmrg/bin/activate`
3. Install Jupyter notebook: `python -m pip install notebook`
4. Update relevant parameters; see [General Parameters](#general-parameters) and [DMRG Parameters](#dmrg-parameters.)
5. Run this notebook. Running in VS Code is recommended, but not necessary.

In [1]:
%reset -f 
# Install pip packages in the current Jupyter kernel (from https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/)
import sys
# Install DMRG
!{sys.executable} -m pip install --extra-index-url=https://block-hczhai.github.io/block2-preview/pypi/ git+https://github.com/jtcantin/dmrghandler

!{sys.executable} -m pip install paramiko h5py numpy

# This took about 8 minutes on my machine for the first time, but only about 7 seconds after that.

Looking in indexes: https://pypi.org/simple, https://block-hczhai.github.io/block2-preview/pypi/
  Cloning https://github.com/jtcantin/dmrghandler to /tmp/pip-req-build-aj4r2op6
  Running command git clone --filter=blob:none --quiet https://github.com/jtcantin/dmrghandler /tmp/pip-req-build-aj4r2op6
  Resolved https://github.com/jtcantin/dmrghandler to commit 367f9c1745b7489937ee9308b33076944d72e54f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import urllib.request
from pathlib import Path
import json
import dmrghandler.config_io as config_io
import dmrghandler.slurm_scripts as slurm_scripts
import h5py
import jsonschema
import run_support as rs

# General Parameters

In [3]:
remove_temp_files = True # IMPORTANT: If you want to keep the DRMG output files, not just the jsons, set this to False
environment_path=Path("../env_run_dmrg")
ppk_path="/mnt/c/darpa-qb-key.ppk" # Update this to the path of your private key
sftp_username="darpa-qb" # Update this to your username
local_store_path = Path("dmrg_calculation_storage")
repository_url = "https://github.com/isi-usc-edu/qb-gsee-benchmark/archive/refs/heads/main.zip"

solution_save_location = "solution_jsons"
problem_instance_files_repository_path = (
    "problem_instances"
)

chosen_problem_instance_files = [
    # "problem_instance.mn_mono.cb40f3f7-ffe8-40e8-4544-f26aad5a8bd8.json",
    # "problem_instance.planted_solution_0007.3aaf0bb4-b412-4746-922a-c380b4024d00.json",
    # "problem_instance.planted_solution_0010.bf3fb654-72c7-4fd4-bfd1-9ffef5aaebd8.json",
    "problem_instance.planted_solution_0001.3d3b9e8a-7842-4ca1-bd82-eddb9804972d.json",
    "problem_instance.planted_solution_0008.b57eb979-5c09-4974-97b4-5862e109a1ae.json",
]


json_solution_schema_url = "https://raw.githubusercontent.com/isi-usc-edu/qb-gsee-benchmark/main/schemas/solution.schema.0.0.1.json"
json_solution_schema_url_file = json_solution_schema_url

contact_info = [{
    "name": "Example E. Example",
    "email": "example@example.ca",
    "institution": "University of Example",
}]
compute_details = {
    "computing_environment_name": "Example Computer",
    "cpu_description": 'Intel i5-1135G7 @ 2.40GHz',
    "ram_available_gb": "16GB",
    "clock_speed": "2.4 GHz",
    "total_num_cores" : 4,
}

solver_details = {
"solver_uuid":"13474cee-e648-48d3-9526-0314533ae30d",
"solver_short_name":"DMRG_surface_lowest_energy",
"compute_hardware_type":"classical_computer",
"classical_hardware_details":compute_details,
"algorithm_details":"DMRG with the lowest variational energy obtained so far.",
"software_details":"Block2 v0.5.3rc16 with dmrghandler, commit version d603fdc6409fc194a416aa3a519362d5d91790d9 or later.",
}

### Obtain Problem Instance and FCIDUMP Files

In [4]:
# Download problem instance files
repository_filepath = Path("repository.zip")

repository_path = Path("qb-gsee-benchmark-main")
if not repository_path.exists():
    # Download repository
    urllib.request.urlretrieve(repository_url, repository_filepath.name)
    # unzip repository
    os.system(f"unzip {repository_filepath}")

problem_instance_files_path = repository_path / problem_instance_files_repository_path
problem_instance_files = list(problem_instance_files_path.glob("*.json"))

In [5]:
# Download and save FCIDUMPs as listed in chosen_problem_instance_files
data_file_list = rs.download_task_fcidump_files(
    chosen_problem_instance_files,
    problem_instance_files_path,
    local_store_path,
    ppk_path,
    sftp_username,
)
print(data_file_list)

File fetched successfully from sftp.l3harris.com: fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80.gz -> dmrg_calculation_storage/fcidumps/fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80.gz
Uncompressing fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80.gz
File fetched successfully from sftp.l3harris.com: fcidump.planted_solution_0008.44060124-8f2d-4a91-80a9-5a29f0f4f716.gz -> dmrg_calculation_storage/fcidumps/fcidump.planted_solution_0008.44060124-8f2d-4a91-80a9-5a29f0f4f716.gz
Uncompressing fcidump.planted_solution_0008.44060124-8f2d-4a91-80a9-5a29f0f4f716.gz
[PosixPath('dmrg_calculation_storage/fcidumps/fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80'), PosixPath('dmrg_calculation_storage/fcidumps/fcidump.planted_solution_0008.44060124-8f2d-4a91-80a9-5a29f0f4f716')]


# DMRG Parameters 

In [6]:
# DMRG parameters for dmrghanlder
# dmrghandler is a wrapper for Block2: https://block2.readthedocs.io/en/latest/#

# To simply run the script, you only need to update the following:
#       num_threads_list              # Ensure this is not more than the number of cores (or threads) on your machine
#       n_mkl_threads_list            # Ensure this is not more than the number of cores (or threads) on your machine
#       stack_mem                     # Ensure this is less than the total memory available on your machine; not a hard memory limit on Block2, though
#       python_environment_location   # Update this to the path of your python environment where this notebook is running

# For more control of the calculation, consider also changing:
#       max_time_limit_sec_list
#       starting_bond_dimension_list
#       max_num_sweeps_list
#       sweep_schedule_bond_dims_parameters
#       sweep_schedule_noise_list
#       sweep_schedule_davidson_threshold_list
#       init_state_seed_list
#       symmetry_type_list
#       reordering_method_list
#       config_file_prefix
#       job_name

# For all list parameters, if a list with more than one value is given, the length of the list
# must be the same as the length of data_file_list
# If a single value is given, it will be used for all FCIDUMP files
config_dict = {
    "plot_filename_prefix_list": [
        Path(fcidump_file).stem for fcidump_file in data_file_list
    ],
    "main_storage_folder_path_prefix": str(local_store_path),
    "max_bond_dimension_list": [10],
    "max_time_limit_sec_list": [5 * 60],  # Max time limit per FCIDUMP file in seconds
    "min_energy_change_hartree_list": [
        1e-4
    ],  # Convergence threshold as the bond dimension is increased
    "extrapolation_type_list": ["discard_weights"],  # Auto extrapolation, unreliable
    "starting_bond_dimension_list": [4],
    "max_num_sweeps_list": [20],
    "energy_convergence_threshold_list": [1e-8],
    "sweep_schedule_bond_dims_parameters": [
        [(2, 4), (1, 5)]
    ],  # (division_factor, count),
    # e.g. [(2, 4), (1, 5)] and bond dimension of 3 -> [1, 1, 1, 1, 3, 3, 3, 3, 3]
    "sweep_schedule_noise_list": [[1e-4] * 4 + [1e-5] * 4 + [0]],
    "sweep_schedule_davidson_threshold_list": [[1e-10] * 9],
    "init_state_bond_dimension_division_factor_list": [2],
    "init_state_seed_list": [
        658724
    ],  # Random number generator seed for choosing the initial MPS state
    "initial_mps_method_list": ["random"],
    "factor_half_convention_list": [True],  # True for standard FCIDUMP files
    "symmetry_type_list": ["SU(2)"],  # "SZ" or "SU(2)"
    "num_threads_list": [4],
    "n_mkl_threads_list": [4],
    "track_mem": [False],
    "reordering_method_list": ["fiedler, interaction matrix"],
    "calc_v_score_bool_list": [True],
}

dmrg_advanced_config = {
    "occupancy_hint": None,
    "full_fci_space_bool": True,
    "init_state_direct_two_site_construction_bool": False,
    "davidson_type": None,  # Default is None, for "Normal"
    "eigenvalue_cutoff": 1e-20,  # Cutoff of eigenvalues, default is 1e-20
    "davidson_max_iterations": 4000,  # Default is 4000
    "davidson_max_krylov_subspace_size": 50,  # Default is 50
    "lowmem_noise_bool": False,  # Whether to use a lower memory version of the noise, default is False
    "sweep_start": 0,  # Default is 0, where to start sweep
    "initial_sweep_direction": None,  # Default is None, True means forward sweep (left-to-right)
    "stack_mem": 10
    * 1024
    * 1024
    * 1024,  # in bytes; ensure that this value is less than the total memory available
    "stack_mem_ratio": 0.9,
    # "do_single_calc": False,
    "num_states": 1,  # Number of states to calculate, default is 1, the ground state
}

# Generate configuration files
config_files_list, config_dict_single_file_list = config_io.gen_config_files(
    data_file_list=data_file_list,
    config_dict=config_dict,
    dmrg_advanced_config=dmrg_advanced_config,
    config_file_prefix="dmrg_example_run_",
)
print(f"config_files_list: {config_files_list}")
# print(f"config_dict_single_file_list: {config_dict_single_file_list}")

# Parameters for when using SLURM on a cluster
submit_dict = {
    "time_cap_string": "00-23:59:00",
    "job_name": "dmrg_example_run_",
    "email": "eample_email@example.com",
    "account_name": "example",
    "tasks_per_node": "1",
    "cpus_per_task": "40",
    "partition": "debug",
    "python_environment_location": "../env_run_dmrg",
}

# Generate python and SLURM submission scripts
slurm_scripts.gen_run_files(submit_dict, config_dict_single_file_list)


# submit_commands only for use on a cluster with SLURM installed
submit_commands = slurm_scripts.gen_submit_commands(config_dict_single_file_list)
print(f"submit_commands: \n{submit_commands}")

config_files_list: [PosixPath('config_store/6ff48658-78f6-408d-919f-0277071eb641/dmrg_example_run_0_6ff48658-78f6-408d-919f-0277071eb641.yaml'), PosixPath('config_store/0c512ace-3ea6-4b01-b8d0-5de0eb4760c9/dmrg_example_run_1_0c512ace-3ea6-4b01-b8d0-5de0eb4760c9.yaml')]
submit_commands: 
# dmrg_calculation_storage/fcidumps/fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80
sbatch ../6ff48658-78f6-408d-919f-0277071eb641/submit_6ff48658-78f6-408d-919f-0277071eb641.sh
# dmrg_calculation_storage/fcidumps/fcidump.planted_solution_0008.44060124-8f2d-4a91-80a9-5a29f0f4f716
sbatch ../0c512ace-3ea6-4b01-b8d0-5de0eb4760c9/submit_0c512ace-3ea6-4b01-b8d0-5de0eb4760c9.sh



cd config_store/submit_dir
sbatch ../6ff48658-78f6-408d-919f-0277071eb641/submit_6ff48658-78f6-408d-919f-0277071eb641.sh
sbatch ../0c512ace-3ea6-4b01-b8d0-5de0eb4760c9/submit_0c512ace-3ea6-4b01-b8d0-5de0eb4760c9.sh




### Run DMRG

In [7]:
# Run DMRG on the FCIDUMPs
scratch_sim_path = Path(local_store_path) / Path("scratch_sim")
scratch_sim_path.mkdir(parents=True, exist_ok=True)
scratch_sim_path_absolute = scratch_sim_path.resolve()
dmrg_hdf5_files = []
for config_dict in config_dict_single_file_list:

    data_config = config_dict["data_config"]
    python_run_file_name = data_config["python_run_file"]
    os.environ["SCRATCH"] = str(scratch_sim_path_absolute)

    # Run DMRG
    os.system(f"{str(environment_path)}/bin/python {python_run_file_name}")
    print("DMRG NOW EXITED")

    # Get results
    main_storage_folder_path = data_config["main_storage_folder_path"]
    hdf5_file_path = Path(main_storage_folder_path) / Path("dmrg_results.hdf5")
    dmrg_hdf5_files.append(hdf5_file_path)

    with h5py.File(hdf5_file_path, "r") as f:
        dmrg_energies = f["/final_dmrg_results/past_energies_dmrg"][:]
        dmrg_bond_dimensions = f["/final_dmrg_results/bond_dims_used"][:]
        discarded_weights = f["/final_dmrg_results/past_discarded_weights"][:]

        h_min_e_optket_norm = float(
            f["/first_preloop_calc/dmrg_results/h_min_e_optket_norm"][()]
        )
        variance = float(f["/first_preloop_calc/dmrg_results/optket_variance"][()])
        v_score_numerator = float(
            f["/first_preloop_calc/dmrg_results/v_score_numerator"][()]
        )
        deviation_init_ket = float(
            f["/first_preloop_calc/dmrg_results/deviation_init_ket"][()][0]
        )
        v_score_init_ket = float(
            f["/first_preloop_calc/dmrg_results/v_score_init_ket"][()][0]
        )
        hf_energy = float(f["/first_preloop_calc/dmrg_results/hf_energy"][()])
        deviation_hf = float(f["/first_preloop_calc/dmrg_results/deviation_hf"][()][0])
        v_score_hartree_fock = float(
            f["/first_preloop_calc/dmrg_results/v_score_hartree_fock"][()][0]
        )
        initial_ket_energy = float(
            f["/first_preloop_calc/dmrg_results/initial_ket_energy"][()]
        )

    print(f"dmrg_energies: {dmrg_energies}")
    print(f"dmrg_bond_dimensions: {dmrg_bond_dimensions}")
    print(f"discarded_weights: {discarded_weights}")
    print(f"h_min_e_optket_norm: {h_min_e_optket_norm}")
    print(f"variance: {variance}")
    print(f"v_score_numerator: {v_score_numerator}")
    print(f"deviation_init_ket: {deviation_init_ket}")
    print(f"v_score_init_ket: {v_score_init_ket}")
    print(f"hf_energy: {hf_energy}")
    print(f"deviation_hf: {deviation_hf}")
    print(f"v_score_hartree_fock: {v_score_hartree_fock}")
    print(f"initial_ket_energy: {initial_ket_energy}")

Parsing config_store/6ff48658-78f6-408d-919f-0277071eb641/fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80
integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =      21427

Build MPO | Nsites =    14 | Nterms =      21427 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    14 .. Mmpo =    13 DW = 0.00e+00 NNZ =       13 SPT = 0.0000 Tmvc = 0.001 T = 0.004
 Site =     1 /    14 .. Mmpo =    46 DW = 0.00e+00 NNZ =       54 SPT = 0.9097 Tmvc = 0.001 T = 0.004
 Site =     2 /    14 .. Mmpo =    72 DW = 0.00e+00 NNZ =      705 SPT = 0.7871 Tmvc = 0.001 T = 0.005
 Site =     3 /    14 .. Mmpo =   102 DW = 0.00e+00 NNZ =      862 SPT = 0.8826 Tmvc = 0.001 T = 0.005
 Site =     4 /    14 .. Mmpo =   140 DW = 0.00e+00 NNZ =     1291 SPT = 0.9096 Tmvc = 0.002 T = 0.006
 Site =     5 /    14 .. Mmpo =   186 DW = 0.00e+00 NNZ =     1731 SPT = 0.9335 Tmvc = 0.002 T = 0.006
 Site =     6 /    14 .. Mmpo =   240 DW = 0.00e+00 NNZ =     2131 SPT = 0.9523 

/home/jtcantin/utoronto/gsee_benchmark_mods/qb-gsee-benchmark/env_run_dmrg/lib/python3.10/site-packages/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =      61054

Build MPO | Nsites =    14 | Nterms =      61054 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    14 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.003 T = 0.008
 Site =     1 /    14 .. Mmpo =    98 DW = 0.00e+00 NNZ =      595 SPT = 0.7665 Tmvc = 0.003 T = 0.013
 Site =     2 /    14 .. Mmpo =   142 DW = 0.00e+00 NNZ =     1323 SPT = 0.9049 Tmvc = 0.003 T = 0.012
 Site =     3 /    14 .. Mmpo =   202 DW = 0.00e+00 NNZ =     2355 SPT = 0.9179 Tmvc = 0.004 T = 0.019
 Site =     4 /    14 .. Mmpo =   278 DW = 0.00e+00 NNZ =     3531 SPT = 0.9371 Tmvc = 0.005 T = 0.021
 Site =     5 /    14 .. Mmpo =   370 DW = 0.00e+00 NNZ =     4743 SPT = 0.9539 Tmvc = 0.006 T = 0.021
 Site =     6 /    14 .. Mmpo =   478 DW = 0.00e+00 NNZ =     5883 SPT = 0.9667 Tmvc = 0.006 T = 0.019
 Site =     7 /    14 .. Mmpo =   370 DW = 0.00e+00 NNZ =    27699 SPT = 0.8434 Tmvc = 0.007 T = 0.034

/home/jtcantin/utoronto/gsee_benchmark_mods/qb-gsee-benchmark/env_run_dmrg/lib/python3.10/site-packages/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =      61054

Build MPO | Nsites =    14 | Nterms =      61054 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    14 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.003 T = 0.008
 Site =     1 /    14 .. Mmpo =    98 DW = 0.00e+00 NNZ =      595 SPT = 0.7665 Tmvc = 0.003 T = 0.010
 Site =     2 /    14 .. Mmpo =   142 DW = 0.00e+00 NNZ =     1323 SPT = 0.9049 Tmvc = 0.003 T = 0.013
 Site =     3 /    14 .. Mmpo =   202 DW = 0.00e+00 NNZ =     2355 SPT = 0.9179 Tmvc = 0.004 T = 0.014
 Site =     4 /    14 .. Mmpo =   278 DW = 0.00e+00 NNZ =     3531 SPT = 0.9371 Tmvc = 0.005 T = 0.016
 Site =     5 /    14 .. Mmpo =   370 DW = 0.00e+00 NNZ =     4743 SPT = 0.9539 Tmvc = 0.005 T = 0.017
 Site =     6 /    14 .. Mmpo =   478 DW = 0.00e+00 NNZ =     5883 SPT = 0.9667 Tmvc = 0.006 T = 0.021
 Site =     7 /    14 .. Mmpo =   370 DW = 0.00e+00 NNZ =    27699 SPT = 0.8434 Tmvc = 0.007 T = 0.034

...

integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =      61054

Build MPO | Nsites =    14 | Nterms =      61054 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    14 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.002 T = 0.009
 Site =     1 /    14 .. Mmpo =    98 DW = 0.00e+00 NNZ =      595 SPT = 0.7665 Tmvc = 0.004 T = 0.011
 Site =     2 /    14 .. Mmpo =   142 DW = 0.00e+00 NNZ =     1323 SPT = 0.9049 Tmvc = 0.003 T = 0.012
 Site =     3 /    14 .. Mmpo =   202 DW = 0.00e+00 NNZ =     2355 SPT = 0.9179 Tmvc = 0.004 T = 0.014
 Site =     4 /    14 .. Mmpo =   278 DW = 0.00e+00 NNZ =     3531 SPT = 0.9371 Tmvc = 0.004 T = 0.016
 Site =     5 /    14 .. Mmpo =   370 DW = 0.00e+00 NNZ =     4743 SPT = 0.9539 Tmvc = 0.006 T = 0.020
 Site =     6 /    14 .. Mmpo =   478 DW = 0.00e+00 NNZ =     5883 SPT = 0.9667 Tmvc = 0.007 T = 0.021
 Site =     7 /    14 .. Mmpo =   370 DW = 0.00e+00 NNZ =    27699 SPT = 0.8434 Tmvc = 0.007 T = 0.035

/home/jtcantin/utoronto/gsee_benchmark_mods/qb-gsee-benchmark/env_run_dmrg/lib/python3.10/site-packages/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.9467e-03                                    1.25e-01    
       1              2         8.3418e-04      1.11e-03       1.78e-02       3.96e-05    
       2              3         8.3418e-04      1.12e-10       5.66e-06       3.32e-14    
`gtol` termination condition is satisfied.
Function evaluations 3, initial cost 1.9467e-03, final cost 8.3418e-04, first-order optimality 3.32e-14.


/home/jtcantin/utoronto/gsee_benchmark_mods/qb-gsee-benchmark/env_run_dmrg/lib/python3.10/site-packages/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


DMRG NOW EXITED
dmrg_energies: [-123.36638439 -123.38170068 -123.3961879  -123.40299318 -123.40434842
 -123.40793496 -123.41406487]
dmrg_bond_dimensions: [ 4  5  6  7  8  9 10]
discarded_weights: [2.71588498e-04 2.76975135e-05 4.81653711e-05 8.63729953e-05
 2.61163066e-05 4.51685459e-05 5.91700800e-06]
h_min_e_optket_norm: 1.180190252915801
variance: 1.3928490330774623
v_score_numerator: 25.071282595394322
deviation_init_ket: -250.48726564722824
v_score_init_ket: 0.0003995813853198653
hf_energy: -26.66551405073119
deviation_hf: -96.70087033887421
v_score_hartree_fock: 0.0026811171234227614
initial_ket_energy: 127.12088125762286
Parsing config_store/0c512ace-3ea6-4b01-b8d0-5de0eb4760c9/fcidump.planted_solution_0008.44060124-8f2d-4a91-80a9-5a29f0f4f716
integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =      21427

Build MPO | Nsites =    14 | Nterms =      21427 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    14 .. Mmpo =    13 DW = 0.00e+00 NNZ =     

### Collect Solution Data

In [8]:
uuid_fcidump_mapping_dict = rs.get_uuid_fcidump_mapping(submit_commands, local_store_path)
print(uuid_fcidump_mapping_dict)


{'6ff48658-78f6-408d-919f-0277071eb641': 'fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80', '0c512ace-3ea6-4b01-b8d0-5de0eb4760c9': 'fcidump.planted_solution_0008.44060124-8f2d-4a91-80a9-5a29f0f4f716'}


In [9]:
all_dmrgh5_data = rs.collect_dmrg_data(dmrg_hdf5_files, uuid_fcidump_mapping_dict)
all_dmrgh5_data = rs.filter_lowest_energy_data(all_dmrgh5_data)

dmrg_calculation_storage/6ff48658-78f6-408d-919f-0277071eb641/dmrg_results.hdf5
Last loop included = 5
Processed results available
Checking that processed results match raw results.
dmrg_calculation_storage/0c512ace-3ea6-4b01-b8d0-5de0eb4760c9/dmrg_results.hdf5
Last loop included = 3
Processed results available
Checking that processed results match raw results.


In [10]:
prob_inst_data_sol_dict = rs.map_fcidump_to_problem_instances(
    chosen_problem_instance_files, problem_instance_files_path, all_dmrgh5_data
)
print(prob_inst_data_sol_dict)


['problem_instance.planted_solution_0001.3d3b9e8a-7842-4ca1-bd82-eddb9804972d.json', 'problem_instance.planted_solution_0008.b57eb979-5c09-4974-97b4-5862e109a1ae.json']
sftp://sftp.l3harris.com/gsee/planted_solutions/fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80.gz
sftp://sftp.l3harris.com/gsee/planted_solutions/fcidump.planted_solution_0008.44060124-8f2d-4a91-80a9-5a29f0f4f716.gz
{'3d3b9e8a-7842-4ca1-bd82-eddb9804972d': {'prob_inst_file': PosixPath('qb-gsee-benchmark-main/problem_instances/problem_instance.planted_solution_0001.3d3b9e8a-7842-4ca1-bd82-eddb9804972d.json'), 'prob_inst_short_name': 'planted_solution_0001', 'be3812d3-5975-4736-8a23-96d64e50e66a': {'solution_uuid': '6ff48658-78f6-408d-919f-0277071eb641', 'instance_data_object_uuid': '3add17d8-cb86-4ae5-8474-96d272269d80', 'instance_data_object_url': 'sftp://sftp.l3harris.com/gsee/planted_solutions/fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80.gz', 'dmrg_data': {'solution_uuid': '6f

### Save solution json files

In [11]:

rs.create_solution_files(
    prob_inst_data_sol_dict,
    solution_save_location,
    json_solution_schema_url_file,
    contact_info,
    solver_details,
)

rs.validate_solution_files(json_solution_schema_url, prob_inst_data_sol_dict)


dict_keys(['prob_inst_file', 'prob_inst_short_name', 'be3812d3-5975-4736-8a23-96d64e50e66a'])
dict_items([('prob_inst_file', PosixPath('qb-gsee-benchmark-main/problem_instances/problem_instance.planted_solution_0001.3d3b9e8a-7842-4ca1-bd82-eddb9804972d.json')), ('prob_inst_short_name', 'planted_solution_0001'), ('be3812d3-5975-4736-8a23-96d64e50e66a', {'solution_uuid': '6ff48658-78f6-408d-919f-0277071eb641', 'instance_data_object_uuid': '3add17d8-cb86-4ae5-8474-96d272269d80', 'instance_data_object_url': 'sftp://sftp.l3harris.com/gsee/planted_solutions/fcidump.planted_solution_0001.3add17d8-cb86-4ae5-8474-96d272269d80.gz', 'dmrg_data': {'solution_uuid': '6ff48658-78f6-408d-919f-0277071eb641', 'dmrg_energy': -123.41406487085237, 'bond_dimension': 10, 'discarded_weight': 5.917007999472202e-06, 'loop_cpu_time': 20.0118996, 'loop_wall_time': 5.293625107, 'num_sweeps': 15, 'final_sweep_delta_energy': 8.724327926756814e-10, 'reordering_method': b'fiedler, interaction matrix', 'reordering_meth

### Remove Temporary Files

In [12]:
if remove_temp_files:
    os.remove(repository_filepath)
    os.remove("dmrghandler.log")
    os.system(f"rm -r {repository_path}")
    os.system(f"rm -r {scratch_sim_path}")
    os.system(f"rm -r {local_store_path}")
    os.system(f"rm -r config_store")
    os.system(f"rm -r tmp_dir")
    